In [42]:
import random

# Grafo de distancias
distancias = {
    'A': {'B': 7, 'C': 9, 'D': 8, 'E': 20},
    'B': {'A': 7, 'C': 10, 'D': 4, 'E': 11},
    'C': {'A': 9, 'B': 10, 'D': 15, 'E': 5},
    'D': {'A': 8, 'B': 4, 'C': 15, 'E': 17},
    'E': {'A': 20, 'B': 11, 'C': 5, 'D': 17}
}

# Nodos del grafo
nodos = ['A', 'B', 'C', 'D', 'E']

# Función para calcular la longitud de una ruta
def calcular_ruta(camino):
    distancia_total = 0
    for i in range(len(camino) - 1):
        if camino[i] != camino[i + 1]:  # Evitamos distancias de un nodo a sí mismo
            distancia_total += distancias[camino[i]][camino[i + 1]]
    return distancia_total

# Función de aptitud (fitness): cuanto menor es la distancia, mayor es la aptitud
def aptitud(camino):
    return 1 / calcular_ruta(camino)

# Inicialización de la población
def crear_poblacion(tamano_poblacion):
    poblacion = []
    for _ in range(tamano_poblacion):
        # Crear una ruta aleatoria, asegurando que 'A' esté siempre al principio
        resto_nodos = [nodo for nodo in nodos if nodo != 'A']
        random.shuffle(resto_nodos)
        camino = ['A'] + resto_nodos  # 'A' solo al principio
        poblacion.append(camino)
    return poblacion

# Selección por torneo
def seleccion(poblacion):
    torneo = random.sample(poblacion, 3)  # Seleccionamos 3 individuos al azar
    torneo.sort(key=lambda x: aptitud(x), reverse=True)  # Ordenamos por aptitud
    return torneo[0]  # Retornamos el mejor

# Cruzamiento (OX) - A debe quedarse al principio
def cruzar(padre1, padre2):
    # Hacemos un crossover asegurando que 'A' esté al principio
    punto1, punto2 = sorted(random.sample(range(1, len(padre1)), 2))  # Evitamos el primer nodo

    hijo1 = ['A'] * len(padre1)
    hijo2 = ['A'] * len(padre2)

    # Copiar el segmento de padre1 a hijo1
    for i in range(punto1, punto2 + 1):
        hijo1[i] = padre1[i]
        hijo2[i] = padre2[i]

    # Rellenar el resto del hijo1 con los genes de padre2 en el orden que quedan disponibles
    for i in range(1, len(padre1)):
        if hijo1[i] == 'A':  # Evitamos el nodo 'A'
            for j in range(1, len(padre2)):
                if padre2[j] not in hijo1:
                    hijo1[i] = padre2[j]
                    break

    # Rellenar el resto del hijo2 con los genes de padre1 en el orden que quedan disponibles
    for i in range(1, len(padre2)):
        if hijo2[i] == 'A':  # Evitamos el nodo 'A'
            for j in range(1, len(padre1)):
                if padre1[j] not in hijo2:
                    hijo2[i] = padre1[j]
                    break

    return hijo1, hijo2

# Mutación (intercambio de dos nodos, sin tocar 'A')
def mutar(camino):
    i, j = random.sample(range(1, len(camino)), 2)  # Evitamos 'A'
    camino[i], camino[j] = camino[j], camino[i]
    return camino

# Función principal del algoritmo genético
def algoritmo_genetico(tamano_poblacion=50, generaciones=100):
    poblacion = crear_poblacion(tamano_poblacion)

    for generacion in range(generaciones):
        poblacion.sort(key=lambda x: aptitud(x), reverse=True)  # Ordenamos la población por aptitud

        nueva_poblacion = poblacion[:10]  # Mantenemos los mejores 10 individuos

        while len(nueva_poblacion) < tamano_poblacion:
            padre1 = seleccion(poblacion)
            padre2 = seleccion(poblacion)
            hijo1, hijo2 = cruzar(padre1, padre2)
            nueva_poblacion.append(mutar(hijo1))
            nueva_poblacion.append(mutar(hijo2))

        poblacion = nueva_poblacion  # Reemplazamos la población actual por la nueva

    mejor_solucion = min(poblacion, key=lambda x: calcular_ruta(x))
    return mejor_solucion, calcular_ruta(mejor_solucion)

# Ejecutar el algoritmo
mejor_ruta, distancia = algoritmo_genetico()
print("Mejor ruta:", mejor_ruta)
print("Distancia:", distancia)


Mejor ruta: ['A', 'D', 'B', 'C', 'E']
Distancia: 27
